In [1]:
import os
import site

In [2]:
import logging
import warnings

from anomalib.utils.loggers import configure_logger, get_experiment_logger


logger = logging.getLogger("anomalib")
configure_logger(level="ERROR") # "<DEBUG, INFO, WARNING, ERROR>"

To use wandb logger install it using `pip install wandb`


In [3]:
from pytorch_lightning import Trainer, seed_everything

from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.data.utils import TestSplitMode
from anomalib.models import get_model
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks

## Monkey Patch

In [4]:
from MyFeatureExtractor import FeatureExtractor
import anomalib.models.padim.torch_model as ptm
ptm.FeatureExtractor = FeatureExtractor

## Load Config

In [5]:
package_path = site.getsitepackages()[0]
for p in site.getsitepackages():
    if "site-package" in p:
        package_path = p
        break

package_path

'C:\\Users\\takanari\\miniconda3\\envs\\python39_dev_anomalib\\lib\\site-packages'

In [6]:
"""
SET MODEL NAME
"""
model_name = "padim"
config_path = os.path.join(package_path, f"anomalib/models/{model_name}/config.yaml")
config = get_configurable_parameters(model_name=model_name, config_path=config_path)

C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\anomalib\config\config.py:280: UserWarning: config.project.unique_dir is set to False. This does not ensure that your results will be written in an empty directory and you may overwrite files.
  warn(


In [7]:
"""
SET MODEL BACKBONE
"""
config.dataset.train_batch_size = 4
config.dataset.eval_batch_size = 4

dict(config.dataset)

{'name': 'mvtec',
 'format': 'mvtec',
 'path': './datasets/MVTec',
 'category': 'bottle',
 'task': 'segmentation',
 'train_batch_size': 4,
 'eval_batch_size': 4,
 'num_workers': 8,
 'image_size': [256, 256],
 'center_crop': None,
 'normalization': 'imagenet',
 'transform_config': {'train': None, 'eval': None},
 'test_split_mode': 'from_dir',
 'test_split_ratio': 0.2,
 'val_split_mode': 'same_as_test',
 'val_split_ratio': 0.5,
 'tiling': {'apply': False, 'tile_size': None, 'stride': None, 'remove_border_count': 0, 'use_random_tiling': False, 'random_tile_count': 16}}

In [8]:
dict(config.logging)

{'logger': [], 'log_graph': False}

In [9]:
dict(config.metrics)

{'image': ['F1Score', 'AUROC'],
 'pixel': ['F1Score', 'AUROC'],
 'threshold': {'method': 'adaptive', 'manual_image': None, 'manual_pixel': None}}

In [10]:
"""
SET MODEL BACKBONE

REF: https://github.com/JohnnyHopp/PaDiM-EfficientNetV2/blob/master/main.py
"""
# config.model.backbone = "resnet18"
# config.model.layers = ['layer1.-1', 'layer2.-1', 'layer3.-1']

config.model.backbone = "efficientnet_b5" # wide_resnet50_2, resnet18
config.model.layers = ['blocks.1.-1', 'blocks.3.-1', 'blocks.4.-1']
config.model.n_features = 100 # "resnet18": 100, "wide_resnet50_2": 550
dict(config.model)

{'name': 'padim',
 'backbone': 'efficientnet_b5',
 'pre_trained': True,
 'layers': ['blocks.1.-1', 'blocks.3.-1', 'blocks.4.-1'],
 'normalization_method': 'min_max',
 'input_size': [256, 256],
 'n_features': 100}

In [11]:
dict(config.optimization)

{'export_mode': None}

In [12]:
dict(config.project)

{'seed': 42, 'path': 'results\\padim\\mvtec\\bottle\\run', 'unique_dir': False}

In [13]:
dict(config.trainer)

{'enable_checkpointing': True,
 'default_root_dir': 'results\\padim\\mvtec\\bottle\\run',
 'gradient_clip_val': 0,
 'gradient_clip_algorithm': 'norm',
 'num_nodes': 1,
 'devices': 1,
 'enable_progress_bar': True,
 'overfit_batches': 0.0,
 'track_grad_norm': -1,
 'check_val_every_n_epoch': 1,
 'fast_dev_run': False,
 'accumulate_grad_batches': 1,
 'max_epochs': 1,
 'min_epochs': None,
 'max_steps': -1,
 'min_steps': None,
 'max_time': None,
 'limit_train_batches': 1.0,
 'limit_val_batches': 1.0,
 'limit_test_batches': 1.0,
 'limit_predict_batches': 1.0,
 'val_check_interval': 1.0,
 'log_every_n_steps': 50,
 'accelerator': 'auto',
 'strategy': None,
 'sync_batchnorm': False,
 'precision': 32,
 'enable_model_summary': True,
 'num_sanity_val_steps': 0,
 'profiler': None,
 'benchmark': False,
 'deterministic': False,
 'reload_dataloaders_every_n_epochs': 0,
 'auto_lr_find': False,
 'replace_sampler_ddp': True,
 'detect_anomaly': False,
 'auto_scale_batch_size': False,
 'plugins': None,
 'mo

In [14]:
dict(config.visualization)

{'show_images': False,
 'save_images': True,
 'log_images': True,
 'image_save_path': None,
 'mode': 'full'}

## Train

In [15]:
datamodule = get_datamodule(config)
model = get_model(config)
experiment_logger = get_experiment_logger(config)
callbacks = get_callbacks(config)

C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\anomalib\utils\callbacks\__init__.py:142: UserWarning: Export option: None not found. Defaulting to no model export
  warnings.warn(f"Export option: {config.optimization.export_mode} not found. Defaulting to no model export")


In [16]:
model.model

PadimModel(
  (feature_extractor): FeatureExtractor(
    (feature_extractor): EfficientNetFeatures(
      (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNormAct2d(
        48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): SiLU(inplace=True)
      )
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (bn1): BatchNormAct2d(
              48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): SiLU(inplace=True)
            )
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (act1): SiLU(inplace=True)
              (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=

In [17]:
trainer = Trainer(**config.trainer, logger=experiment_logger, callbacks=callbacks)
logger.info("Training the model.")
trainer.fit(model=model, datamodule=datamodule)

C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `ROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:613: UserWarning: Checkpoint directory C:\Users\takanari\Documents\work\dev_anomalib_with_docker\work\results\padim\mvtec\bottle\run\weights\lightning exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\pytorch_lightning\core\optimizer.py:183: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\pytorch_lightning\loops\optimization\optimizer_loop.py:138: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")


Validation: 0it [00:00, ?it/s]

# Test

In [18]:
weight_file_path = trainer.checkpoint_callback.best_model_path
weight_file_path

'C:\\Users\\takanari\\Documents\\work\\dev_anomalib_with_docker\\work\\results\\padim\\mvtec\\bottle\\run\\weights\\lightning\\model-v3.ckpt'

In [19]:
logger.info("Loading the best model weights.")
load_model_callback = LoadModelCallback(weights_path=weight_file_path)
trainer.callbacks.insert(0, load_model_callback)

_=trainer.test(model=model, datamodule=datamodule)


Testing: 0it [00:00, ?it/s]

─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       image_AUROC          0.9785714149475098
      image_F1Score         0.9523809552192688
       pixel_AUROC          0.9259722232818604
      pixel_F1Score         0.5058547258377075
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'pixel_F1Score': 0.5058547258377075,
  'pixel_AUROC': 0.9259722232818604,
  'image_F1Score': 0.9523809552192688,
  'image_AUROC': 0.9785714149475098}]

## Appendix

In [20]:
# import timm

# timm.create_model(
#     "efficientnet_b5",
#     pretrained=True,
#     features_only=True,
#     exportable=True,
# )

EfficientNetFeatures(
  (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct